In [ ]:
import pandas as pd
import json
from dateutil.parser import parse
from collections import defaultdict
from tqdm import tqdm
from gensim.models import Word2Vec

In [ ]:
interaction_path = '/home/DATA1/lvxiaoling/linzhongtan/2022fall_anime_recommendation/preprocessed_data/interaction.tsv'
interaction = pd.read_csv(interaction_path,sep='\t')

In [ ]:
interaction_time = interaction['updated_at'].tolist()
new_time = []
for i in range(len(interaction_time)):
    if i%1000 == 0:
        print(i)
    new_time.append(parse(interaction_time[i]))
interaction['updated_at'] = new_time
interaction = interaction.sort_values('updated_at')
selected = interaction[(pd.Timestamp(2020,1,1)<=interaction['updated_at']) & (interaction['updated_at']<=pd.Timestamp(2022,3,31))]

In [ ]:
user_ids = selected['user_id'].tolist()
item_ids = selected['subject_id'].tolist()
user_dict = defaultdict(int)
item_dict = defaultdict(int)
for i in range(len(selected)):
    if i%1000000 == 0:
        print(i)
    user_id = user_ids[i]
    item_id = item_ids[i]
    if user_id not in user_dict:
        user_dict[user_id] = len(user_dict)
    if item_id not in user_dict:
        item_dict[item_id] = len(item_dict)

In [ ]:
selected['user_id'] = selected['user_id'].apply(lambda x:user_dict[x])
selected['subject_id'] = selected['subject_id'].apply(lambda x:item_dict[x]+len(user_dict))
train = selected[selected['updated_at']<pd.Timestamp(2022,1,1)]
test = selected[pd.Timestamp(2022,1,1)<=selected['updated_at']]
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [ ]:
save_path = '/home/DATA1/xiweicheng/STAM/data/anime'
with open(os.path.join(save_path,'train.txt'), 'w') as T:
    for i in range(len(train)):
        T.write(str(train.loc[i,'user_id']) + '\t' + str(train.loc[i,'subject_id']) + '\n')

with open(os.path.join(save_path,'test.txt'), 'w') as T:
    for i in range(len(test)):
        T.write(str(test.loc[i,'user_id']) + '\t' + str(test.loc[i,'subject_id']) + '\n')

In [ ]:
selected['tags'] = selected['tags'].apply(lambda x:x.strip('[').strip(']').replace('\'',''))
gb = selected.groupby('user_id')
tags = gb.apply(lambda g: ','.join(g.tags))
user_tags = tags.apply(lambda x:list(set(x.split(','))))
for i in user_tags:
    if '' in i:
        i.remove('')

selected['tags'] = selected['tags'].apply(lambda x:x.strip('[').strip(']').replace('\'',''))
gb = selected.groupby('subject_id')
tags = gb.apply(lambda g: ','.join(g.tags))
item_tags = tags.apply(lambda x:list(set(x.split(','))))
for i in item_tags:
    if '' in i:
        i.remove('')

In [ ]:
all_tags = user_tags.append(item_tags)
tags = np.concatenate(all_tags.values)
tags = [[str(s).strip()] for s in tags]

In [ ]:
model = Word2Vec(sentences=tags, vector_size=100, min_count=1, workers=4)
tags_embeddings = {}
for t in tags:
    tags_embeddings[t[0]] = model.wv[t[0]].tolist()

In [ ]:
with open('data/anime/tags_embedding.json','w') as f:
    f.write(json.dumps(tags_embeddings))

In [ ]:
user_embeddings = []
for idx, tags in enumerate(user_tags):
    ebs = []
    for t in tags:
        ebs.append(tags_embeddings[t.strip()])
    user_embeddings.append(np.mean(ebs, axis=0).tolist())

item_embeddings = []
for idx, tags in enumerate(item_tags):
    ebs = []
    for t in tags:
        ebs.append(tags_embeddings[t.strip()])
    item_embeddings.append(np.mean(ebs, axis=0).tolist())

In [ ]:
with open('data/anime/embeddings.json', 'w') as f:
    f.write(json.dumps({'user':user_embeddings, 'item':item_embeddings}))